
# Machine Learning Based Recommendation Systems
##  Model-based Collaborative Filtering Systems
## SVD Matrix Factorization

In [1]:
import numpy as np
import pandas as pd

import sklearn
from sklearn.decomposition import TruncatedSVD

The MovieLens dataset was collected by the GroupLens Research Project at the University of Minnesota. You can download the dataset for this demostration at the following URL: https://grouplens.org/datasets/movielens/100k/

### Preparing the data

In [2]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
frame = pd.read_csv('ml-100k/u.data', sep='\t', names=columns)
frame.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
frame.describe()

,user_id,item_id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,1.000000e+05
mean,462.48475,425.530130,3.529860,8.835289e+08
std,266.61442,330.798356,1.125674,5.343856e+06
min,1.00000,1.000000,1.000000,8.747247e+08
25%,254.00000,175.000000,3.000000,8.794487e+08
50%,447.00000,322.000000,4.000000,8.828269e+08
75%,682.00000,631.000000,4.000000,8.882600e+08
max,943.00000,1682.000000,5.000000,8.932866e+08


In [4]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('ml-100k/u.item', sep='|', names=columns, encoding='latin-1')
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

,item_id,movie title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [6]:
movies.head()

,item_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Childrens,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
combined_movies_data = pd.merge(frame, movie_names, on='item_id')
combined_movies_data.head()

,user_id,item_id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [8]:
combined_movies_data.groupby('item_id')['rating'].count().sort_values(ascending=False).head()

item_id
50     583
258    509
100    508
181    507
294    485
Name: rating, dtype: int64

In [9]:
filter = combined_movies_data['item_id']==50
combined_movies_data[filter]['movie title'].unique()

array(['Star Wars (1977)'], dtype=object)

### Building a Utility Matrix

In [11]:
utility_matrix = pd.pivot_table(data=combined_movies_data, values='rating', index='user_id', columns='movie title', fill_value=0)
utility_matrix.head()


movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,2,5,0,0,3,4,0,0,...,0,0,0,5,3,0,0,0,4,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,2,0,0,0,0,4,0,0,...,0,0,0,4,0,0,0,0,4,0


### Transposing the Matrix

In [15]:
utility_matrix.shape

(943, 1664)

In [16]:
X = utility_matrix.T
X.shape

(1664, 943)

### Decomposing the Matrix

In [19]:
SVD = TruncatedSVD(n_components=12, random_state=17)
resultant_matrix = SVD.fit_transform(X)
resultant_matrix.shape

[[  1.03999361e+00   6.59884498e-01   4.56894384e-02 ...,  -7.53561261e-01
   -3.01529677e-01  -5.36511447e-01]
 [  4.36584337e-01  -2.57261460e-01   3.52955096e-01 ...,  -2.47432271e-01
    3.52606851e-01  -8.35835521e-02]
 [  1.25437438e+01   5.66923364e+00  -4.90781163e+00 ...,   3.80234835e+00
    6.02188649e-01  -8.31986802e-01]
 ..., 
 [  3.58929614e-01   3.71257163e-01   2.29745164e-02 ...,  -8.87474913e-02
    1.83072132e-01  -2.97030810e-02]
 [  1.42428013e+00   8.14939513e-01  -4.90237341e-01 ...,   1.56784300e-01
    5.69624249e-01  -6.37080308e-01]
 [  2.29210339e-01  -6.22518604e-03   2.73162116e-01 ...,   8.32677774e-02
   -8.14794217e-02  -1.56028182e-01]]


In [21]:
print(resultant_matrix[0])

[ 1.03999361  0.6598845   0.04568944  0.81444724  0.16747427 -0.98316366
 -0.3198289   0.3067836  -0.03084219 -0.75356126 -0.30152968 -0.53651145]


### Generating a Correlation Matrix

In [33]:
corr_matrix = np.corrcoef(resultant_matrix)
corr_matrix.shape

(1664, 1664)

(1664, 1664)

### Isolating Star Wars From the Correlation Matrix

In [45]:
movies_names = utility_matrix.columns
movies_names
movies_list = list(movies_names)
movies_list
star_wars = movies_list.index('Star Wars (1977)')

1398

In [46]:
corr_star_wars = corr_matrix[1298]
corr_star_wars.shape

(1664,)

### Recommending a Highly Correlated Movie

In [48]:
list(movies_names[(corr_star_wars<1.0) & (corr_star_wars>0.9)])

["Brother's Kiss, A (1997)",
 'Dark City (1998)',
 'Deep Rising (1998)',
 'Excess Baggage (1997)',
 'For Ever Mozart (1996)',
 'Half Baked (1998)',
 'Home Alone 3 (1997)',
 'Innocent Sleep, The (1995)',
 'MURDER and murder (1996)',
 'MatchMaker, The (1997)',
 'Money Talks (1997)',
 'Next Step, The (1995)',
 'Nightwatch (1997)',
 'Nothing Personal (1995)',
 'Phantoms (1998)',
 'Replacement Killers, The (1998)',
 'Ripe (1996)',
 'Rough Magic (1995)',
 'Simple Wish, A (1997)',
 'Sphere (1998)',
 'Spice World (1997)',
 'Tainted (1998)',
 'Wedding Bell Blues (1996)']

In [50]:
list(movies_names[(corr_star_wars<1.0) & (corr_star_wars>0.95)])

['Simple Wish, A (1997)']